In [255]:
import os

In [256]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'

In [257]:
os.chdir("../")

In [258]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml'

In [259]:
proj_link = 'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'
os.chdir(proj_link)

In [260]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path: Path
    model_path: Path
    all_params : dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    






In [261]:
from clientClassifier.constants import *    
from clientClassifier.utils.common import read_yaml, create_directories, save_json

In [262]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config['artifacts_root']])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config['model_evaluation']
        params = self.params['Logistic_Regression']
        schema = self.schema['TARGET_COLUMN']
        
        create_directories([config['root_dir']])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config['root_dir']),
            test_data_path=Path(config['test_data_path']),
            model_path=Path(config['model_path']),
            all_params=params,
            metric_file_name=Path(config['metric_file_name']),
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/richardmukechiwa/client-subscription-prediction.mlflow"
        )
        
        return model_evaluation_config
                
          

In [263]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
import mlflow 



In [ ]:
 # Import mlflow to fix the "mlflow is not defined" error

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred) 
        return rmse, mae, r2
                
                
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)  
        model = joblib.load(self.config.model_path) 
        
        test_x = test_data.drop(self.config.target_column, axis=1)  # Fixed extra parenthesis
        test_y = test_data[self.config.target_column]
        
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():
            
            predicted_qualities  = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)    
            
            #Saving the metrics as local
            scores   = {"rmse": rmse, "mae": mae, "r2": r2}
            
            save_json(path = Path(self.config.metric_file_name), data = scores)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            if tracking_url_type_store != "file":
                
                mlflow.sklearn.log_model(model, "model", registered_model_name = "LogisticRegressionModel")
            else:
                mlflow.sklearn.log_model(model, "model")
        
        
        
        
  

In [265]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)  # Fixed variable name
    model_evaluation.log_into_mlflow()  
except Exception as e:
    raise e


        

[2025-04-16 23:11:27,986: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-16 23:11:27,990: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 23:11:27,995: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-16 23:11:27,996: INFO: common: created directory at: artifacts]
[2025-04-16 23:11:27,998: INFO: common: created directory at: artifacts/model_evaluation]


AttributeError: module 'mlflow' has no attribute 'set_registry_uri'